<a href="https://colab.research.google.com/github/Vaibhavi-Sita/StudyAid/blob/main/StudyAid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install cdqa

In [2]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.converters import pdf_converter
from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline
from cdqa.utils.download import download_model

/usr/local/lib/python3.7/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
# Download model
download_model(model='bert-squad_1.1', dir='./models')

In [ ]:
#!pip install --upgrade pandas==0.25.0

  Using cached pandas-0.25.0-cp37-cp37m-win_amd64.whl (9.2 MB)

In [ ]:
#!pip uninstall numpy
#!pip install --upgrade numpy==1.19.5

^C


In [ ]:
#import numpy
#print(numpy.__version__)

1.21.5


In [5]:
df = pdf_converter(directory_path='./data/pdfs/')
df.head()

2021-12-23 04:45:27,896 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar to /tmp/tika-server.jar.
2021-12-23 04:45:28,486 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server/1.19/tika-server-1.19.jar.md5 to /tmp/tika-server.jar.md5.
2021-12-23 04:45:29,153 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


,title,paragraphs
0,Chapter 9 - Mobile Internet Devices and the Cloud,"[235 , Chapter 9 , Mobile Internet Devices an..."


In [6]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib', max_df=1.0)

# Fit Retriever to documents
cdqa_pipeline.fit_retriever(df=df)

100%|██████████| 231508/231508 [00:00<00:00, 1224876.76B/s]


QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po..._size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=1.0, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

In [7]:
query = 'How many connections to mobile devices are there worldwide?'
prediction = cdqa_pipeline.predict(query)

In [8]:
print('query: {}'.format(query))
print('answer: {}'.format(prediction[0]))
print('title: {}'.format(prediction[1]))
print('paragraph: {}'.format(prediction[2]))

query: How many connections to mobile devices are there worldwide?
answer: over 4 billion
title: Chapter 9 - Mobile Internet Devices and the Cloud
paragraph: 1  estimated that thereare over 4 billion connections to mobile devices worldwide—an astoundingnumber when you realize that this figure represents 60% of the global pop-ulation today. Of course, this does not mean that two out of every threepeople on Earth have a mobile phone. It is common in more than 60 coun-tries, however, for one person to have two or more devices, even while thereare no phones at all in some parts of the globe. In some countries, millionsof people are now experiencing connectivity to the world for the first timethrough wireless technologies. It is changing their economic, social, andpolitical fortunes forevermore. 


train model reader

In [9]:
import os
import torch
import joblib
from cdqa.reader import BertProcessor, BertQA
from cdqa.utils.download import download_squad

In [10]:
download_squad(dir='./data')

In [11]:
train_processor = BertProcessor(do_lower_case=True, is_training=True)
train_examples, train_features = train_processor.fit_transform(X='./data/SQuAD_1.1/train-v1.1.json')

In [ ]:
#!python -m pip install jupyter_contrib_nbextensions

In [ ]:
#!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [ ]:
#!pip install ipywidgets --user

In [12]:
reader = BertQA(train_batch_size=12,
                learning_rate=3e-5,
                num_train_epochs=2,
                do_lower_case=True,
                output_dir='models')

reader.fit(X=(train_examples, train_features))

100%|██████████| 440473133/440473133 [00:16<00:00, 26895815.56B/s]


BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased', do_lower_case=True,
       fp16=False, gradient_accumulation_steps=1, learning_rate=3e-05,
       local_rank=-1, loss_scale=0, max_answer_length=30, n_best_size=20,
       no_cuda=False, null_score_diff_threshold=0.0, num_train_epochs=2,
       output_dir='models', predict_batch_size=8, seed=42, server_ip='',
       server_port='', train_batch_size=12, verbose_logging=False,
       version_2_with_negative=False, warmup_proportion=0.1, warmup_steps=0)

In [13]:
reader.model.to('cpu')
reader.device = torch.device('cpu')

In [14]:
joblib.dump(reader, os.path.join(reader.output_dir, 'bert_qa.joblib'))

['models/bert_qa.joblib']